# Headers

In [1]:
import os
import pandas as pd
import time
import numpy as np
import h5py

# Metrics
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
from utils import train_datapath, test_datapath
from utils import train_val_split

In [ ]:
targets_for_test_df = pd.read_csv('data/targets_for_test.csv', index_col=0)


In [ ]:

def save_submission(test_df, filled_test_predictions, filename='submission.csv'):
    # Create a new DataFrame for the submission
    submission_df = pd.DataFrame({
        'row_id': test_df['row_id'],
        'target': [0] + filled_test_predictions
    })

    # Save the submission file
    submission_df.to_csv(filename, index=False)
    print(f"Predictions saved to {filename}")


# Basic CNN

In [ ]:
train_df = pd.read_csv(train_datapath)
num_features = len(train_df.columns) - 1
del train_df


batch_size, window_size, num_features = 1024, 60, num_features  # dimensions of the dataset

In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F

class BasicCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * (window_size // 2 // 2), 128)  # Adjust based on pooling layers
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * (window_size // 2 // 2))  # Flatten
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


def load_data_for_prediction_batch(h5_file, batch):
    with h5py.File(h5_file, 'r') as h5f:
            inputs = h5f['inputs']  # Load a batch of windows
    return inputs  # Shape will be (batch_size, window_size, num_features)

In [ ]:
train_df = pd.read_csv(train_datapath)



# split the train_df into train and val
X_train, y_train, X_val, y_val = train_val_split(train_df)
X_train['target'] = y_train

train_slice_df = X_train.copy()

X_val['target'] = y_val
val_slice_df = X_val.copy()


input_channels = train_df.shape[1] - '' # Columns minus the target
num_classes = 1  # Binary classification
window_size = 60
batch_size = 1024
train_targets = train_slice_df['target'].values

del train_df, X_train, y_train, X_val, y_val

TypeError: unsupported operand type(s) for -: 'int' and 'str'

In [ ]:
def train_model(model, train_slice_df, train_file, window_size, batch_size, num_epochs=10, learning_rate=0.001):
    # Define loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        batch_idx = 0
        with h5py.File(train_file, 'r') as h5f:
            for start_index in range(0, len(train_targets) - window_size + 1, batch_size):
                # print(f'training batch {batch_idx}')

                # Load a batch of data
                inputs = h5f[f'batch_{batch_idx}'].drop('target')  # Load a batch of windows
                inputs = torch.tensor(inputs, dtype=torch.float32)
                targets = torch.tensor(train_targets[start_index:start_index + batch_size], dtype=torch.float32)

                # Zero the parameter gradients
                optimizer.zero_grad()
                
                print(inputs.shape)

                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                batch_idx += 1
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

start_time = time.time()

# Initialize the model
model = BasicCNN(input_channels, num_classes)
print(model)

# Train the model
train_model(model, train_slice_df, train_file, window_size, batch_size)

train_df = pd.read_csv(train_datapath)
X_train, y_train, X_val, y_val = train_val_split(train_df)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')


# Evaluate the model on validation data in batches
model.eval()
val_predictions = []
val_targets = val_slice_df['target'].values

with torch.no_grad():
    for start_index in range(0, len(val_targets) - window_size + 1, batch_size):
        val_inputs = load_data_for_prediction_batch(val_file, start_index, batch_size, window_size)
        val_inputs = torch.tensor(val_inputs, dtype=torch.float32)
        val_outputs = model(val_inputs)
        batch_predictions = (val_outputs.squeeze().numpy() > 0.5).astype(int)
        val_predictions.extend(batch_predictions)

y_pred = np.array(val_predictions[:len(val_targets)])



# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')

save_submission(X_val, y_pred, filename='submissionCNN.csv')

del train_df, X_train, y_train, X_val, y_val, train_slice_df, val_slice_df


BasicCNN(
  (conv1): Conv1d(10, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=960, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)
torch.Size([1024, 60, 11])


RuntimeError: Given groups=1, weight of size [32, 10, 3], expected input[1024, 60, 11] to have 10 channels, but got 60 channels instead